In [29]:
import pandas as pd
import numpy as np
import networkx as nx

### Building First Bi-Partite Graph

Note that we need to keep values titled "NA" because that's Namibia's ISO2 code. Also, Kosovo doesn't have an ISO3 so we can't use that.

In [30]:
subsidy = pd.read_csv('input/policy/Fossil_Fuel_Subsidies.csv', index_col=0, keep_default_na=False, na_values="")
subsidy.head(3)

,Country,ISO2,ISO3,Indicator,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,Unit,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022,F2023,F2024,F2025
ObjectId,,,,,,,,,,,,,,,,,,,,
1,"Afghanistan, Islamic Rep. of",AF,AFG,Explicit Fossil Fuel Subsidies - Coal,"Parry, Ian; Black, Simon; Vernon, Nate. Septem...",ECGFTEC,Explicit; Coal,"Environment, Climate Change, Government Policy...",Percent of GDP,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Afghanistan, Islamic Rep. of",AF,AFG,Explicit Fossil Fuel Subsidies - Coal,"Parry, Ian; Black, Simon; Vernon, Nate. Septem...",ECGFTEC,Explicit; Coal,"Environment, Climate Change, Government Policy...",US Dollars,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Afghanistan, Islamic Rep. of",AF,AFG,Explicit Fossil Fuel Subsidies - Electricity,"Parry, Ian; Black, Simon; Vernon, Nate. Septem...",ECGFTET,Explicit; Electricity,"Environment, Climate Change, Government Policy...",Percent of GDP,0.0,0.0,0.0,0.083287,0.375834,0.075661,0.252216,0.247982,0.270226,0.271118,0.245831


In [31]:
country_nodes = subsidy.ISO2.unique()
policy_nodes = subsidy.CTS_Code.unique()
print("Number of Countries:", len(country_nodes))
print("Number of Subsidies:", len(policy_nodes))

Number of Countries: 197
Number of Subsidies: 21


Another question would be how do we create groups of subsidies.

In [32]:
B = nx.Graph()
B.add_nodes_from(country_nodes, bipartite=0)
B.add_nodes_from(policy_nodes, bipartite=1)

Note that the following countries do not have the Petroleum field. We'll simply not draw edges.

In [33]:
full = set(subsidy[subsidy.ISO2 == "AF"].CTS_Name)
subsidy_sub = subsidy[subsidy.Unit == "Percent of GDP"]
for country in country_nodes:
    subset = subsidy_sub[subsidy_sub.ISO2 == country]
    if len(subset) < 21:
        print(country, len(subset))
        partial = set(subsidy[subsidy.ISO2 == country].CTS_Name)
        print(full - partial)

AI 20
{'Implicit; Petroleum'}
MH 20
{'Implicit; Petroleum'}
PW 20
{'Implicit; Petroleum'}
SM 20
{'Implicit; Petroleum'}
TV 20
{'Implicit; Petroleum'}


Loop through every country/policy combination and add an edge if the value for specified year was greater than 0.

In [34]:
df_temp = subsidy[subsidy.Unit == "Percent of GDP"]
df_temp.set_index(["ISO2", "CTS_Code"], inplace = True)
edge_list = []
year = "F2018"

for country in country_nodes:
    subset = df_temp.loc[country]
    for policy_code in subset.index:
        weight = subset.loc[policy_code][year]
        if weight > 0:
            edge = (country, policy_code, weight)
            edge_list.append(edge)

B.add_weighted_edges_from(edge_list)

In [35]:
print("Number of Edges:", B.number_of_edges())
print("Number of Possible Edges:", len(country_nodes) * len(policy_nodes))

Number of Edges: 2825
Number of Possible Edges: 4137


### Green Bonds Graph

In [36]:
green_bonds = pd.read_csv("input/policy/Green_Bonds.csv", index_col=0, keep_default_na=False, na_values="")

# Filtered Down to countries
green_bonds = green_bonds[np.logical_not(green_bonds["ISO2"].isna())]

green_bond_country_nodes = green_bonds.ISO2.unique()

# Need to figure out the differences between these bonds
bonds_nodes = green_bonds["CTS_Name"].unique()

Looks likes most years no one invested in bonds. Still very few countries invest in bonds but each year it seems the number increases.

In [37]:
import matplotlib.pyplot as plt

# Get the data
num_countries = green_bonds[green_bonds.columns[12:]].count().array()
years = [int(year[1:]) for year in green_bonds.index]

# Plot
fig,ax = plt.subplots()
ax.scatter(years, num_countries)
ax.set_title("Number of Countries using Green Bonds by year")
ax.set_ylabel("Number of Countries")
ax.set_xlabel("Year")

plt.show()

TypeError: 'PandasArray' object is not callable

Furthermore we can break the graph down into its two different categories

In [ ]:
import matplotlib.pyplot as plt

# Get the data
issuances = green_bonds[green_bonds["CTS_Name"] == "Green Bonds Issuances"]

# Calculate Variables
issuance_count = issuances[issuances.columns[12:]].count()
nonissuance_count = num_countries - issuance_count
y = np.vstack([nonissuance_count, issuance_count])

# Plot
fig,ax = plt.subplots()
# Using years var from last cell
ax.stackplot(years, y, labels = ["Nonissuance Green Bonds", "Issuance Green Bonds",])
ax.set_title("Number of Countries using Green Bonds by year")
ax.set_ylabel("Number of Countries")
ax.set_xlabel("Year")
ax.legend(loc = "upper left")

plt.show()

NameError: name 'num_countries' is not defined

In [ ]:
Bonds = nx.Graph()
Bonds.add_nodes_from(green_bond_country_nodes, bipartite=0)
Bonds.add_nodes_from(bonds_nodes, bipartite=1)

In [44]:
green_bonds_temp = green_bonds.set_index(["ISO2", "CTS_Name"], inplace = False)
green_bonds_temp
edge_list = []
year = "F2015"

for country in green_bond_country_nodes:
    subset = green_bonds_temp.loc[country]
    for bond in subset.index:
        weight = subset.loc[bond][year]
        if weight > 0:
            edge = (country, bond, weight)
            edge_list.append(edge)

Bonds.add_weighted_edges_from(edge_list)

### Environmental Taxes

First lets load in the data and take a quick peak.  The final table is a table about how many measures each country implements.

In [ ]:
taxes = pd.read_csv("input/policy/Environmental_Taxes.csv")

num_countries = len(taxes["Country"].unique())
print(f"There are {num_countries} unique countries that are implementing environmental taxes")
print("Each country typically reports 10 different measures, but there are a few that report less than 10 measures")

taxes.Country.value_counts().value_counts()

There are 124 unique countries that are implementing environmental taxes
Each country typically reports 10 different measures, but there are a few that report less than 10 measures


count
10    118
8       3
6       1
5       1
4       1
Name: count, dtype: int64

In [ ]:
country_nodes = taxes.ISO2.unique()
policy_nodes = taxes.CTS_Code.unique()
print("Number of Countries:", len(country_nodes))
print("Number of Subsidies:", len(policy_nodes))

B = nx.Graph()
B.add_nodes_from(country_nodes, bipartite=0)
B.add_nodes_from(policy_nodes, bipartite=1)

Number of Countries: 124
Number of Subsidies: 5


In [47]:
edges = []
year = "F2018"
taxes_sub = taxes[taxes.Unit == "Percent of GDP"]
for country in country_nodes:
    subset = taxes_sub[taxes_sub.ISO2 == country]
    for policy in policy_nodes:
        weight = subset[subset.CTS_Code == policy][year].values
        if weight > 0:
            edge = (country, policy, weight[0])
            edges.append(edge)

/var/folders/ny/dhq_dcbs49v6dcy_nn3lxbz40000gn/T/ipykernel_24698/2768913331.py:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if weight > 0:


In [ ]:
B.add_weighted_edges_from(edges)
print("Number of Edges:", B.number_of_edges())
print("Number of Possible Edges:", len(country_nodes) * len(policy_nodes))

Number of Edges: 415
Number of Possible Edges: 620


### Environmental Protection Expenditures

In [41]:
expenditures = pd.read_csv("input/policy/Environmental_Protection_Expenditures.csv", index_col=0, keep_default_na=False, na_values="")
expenditures.head(5)

,Country,ISO2,ISO3,Indicator,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,Unit,F1995,...,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021
ObjectId,,,,,,,,,,,,,,,,,,,,,
1,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on biodiversity & landscape protec...,"International Monetary Fund (IMF), Statistics ...",GENB_G14,Expenditure on Biodiversity & Landscape Protec...,"Government and Public Sector Finance, Expendit...",Domestic Currency,NaN,...,NaN,NaN,NaN,NaN,1.243785e+08,1.258269e+09,NaN,NaN,NaN,NaN
2,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on biodiversity & landscape protec...,"International Monetary Fund (IMF), Statistics ...",GENB_G14,Expenditure on Biodiversity & Landscape Protec...,"Government and Public Sector Finance, Expendit...",Percent of GDP,NaN,...,NaN,NaN,NaN,NaN,9.433491e-03,9.134202e-02,NaN,NaN,NaN,NaN
3,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on environment protection,"International Monetary Fund (IMF), Statistics ...",GEN_G14,Environmental Protection Expenditures,"Government and Public Sector Finance, Expendit...",Domestic Currency,NaN,...,2.846995e+09,3.344665e+09,2.204964e+09,1.122601e+09,1.702933e+09,2.377937e+09,NaN,NaN,NaN,NaN
4,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on environment protection,"International Monetary Fund (IMF), Statistics ...",GEN_G14,Environmental Protection Expenditures,"Government and Public Sector Finance, Expendit...",Percent of GDP,NaN,...,2.754469e-01,2.994793e-01,1.863813e-01,9.152359e-02,1.291590e-01,1.726226e-01,NaN,NaN,NaN,NaN
5,"Afghanistan, Islamic Rep. of",AF,AFG,Expenditure on environmental protection n.e.c.,"International Monetary Fund (IMF), Statistics ...",GENO_G14,Expenditure on Environmental Protection Not El...,"Government and Public Sector Finance, Expendit...",Domestic Currency,NaN,...,NaN,NaN,NaN,NaN,9.669078e+07,8.846917e+07,NaN,NaN,NaN,NaN


In [45]:
country_nodes = expenditures.ISO2.unique()
policy_nodes = expenditures.CTS_Code.unique()
print("Number of countries:", len(country_nodes))
print("Number of expenditures:", len(policy_nodes))

E = nx.Graph()
E.add_nodes_from(country_nodes, bipartite=0)
E.add_nodes_from(policy_nodes, bipartite=1)

Number of countries: 129
Number of expenditures: 7


In [48]:
edges = []
year = "F2018"

exp_sub = expenditures[expenditures.Unit == "Percent of GDP"]
for country in country_nodes:
    subset = exp_sub[exp_sub.ISO2 == country]
    for policy in policy_nodes:
        weight = subset[subset.CTS_Code == policy][year].values
        if weight.size > 0 and weight > 0:
            edge = (country, policy, weight[0])
            edges.append(edge)

In [49]:

print("Number of possible edges:", len(country_nodes) * len(policy_nodes))
print("Number of edges:", len(edges))

Number of possible edges: 903
Number of edges: 495


### Analysis

In [ ]:
for comp in nx.connected_components(B):
    print(comp)

{'KH', 'BY', 'QA', 'HU', 'LC', 'GQ', 'ECGFTT', 'HR', 'SR', 'ZW', 'AE', 'ET', 'CD', 'LY', 'IQ', 'SS', 'ECGFTP', 'ECGFTI', 'BO', 'BA', 'RW', 'ECGFTIP', 'GA', 'ECGFTIF', 'NI', 'DO', 'SV', 'ID', 'ECGFT', 'RS', 'BT', 'PS', 'CN', 'UA', 'CR', 'DK', 'SC', 'TG', 'ECGFTEC', 'SY', 'NA', 'NR', 'MK', 'NL', 'IR', 'LU', 'DE', 'CM', 'WS', 'KI', 'DZ', 'CO', 'TO', 'LT', 'TZ', 'GT', 'XK', 'ECGFTET', 'ECGFTIR', 'ECGFTIL', 'CG', 'NE', 'PK', 'IS', 'BD', 'ECGFTEN', 'MY', 'TR', 'ZM', 'MX', 'SG', 'BB', 'KG', 'SE', 'LB', 'ECGFTE', 'CH', 'ML', 'KW', 'BF', 'GB', 'AW', 'MT', 'MZ', 'SO', 'MU', 'NG', 'ECGFTIN', 'AR', 'IL', 'MR', 'MO', 'TW', 'ECGFTIA', 'ECGFTIG', 'AG', 'TL', 'OM', 'IN', 'AF', 'ECGFTIC', 'KM', 'TN', 'IE', 'GY', 'JO', 'ECGFTIW', 'BR', 'MW', 'FM', 'NP', 'AU', 'BN', 'PE', 'TT', 'GD', 'ECGFTIT', 'GR', 'AT', 'AM', 'GE', 'FJ', 'MA', 'VC', 'ECGFTEP', 'BZ', 'CV', 'CF', 'BH', 'CZ', 'NZ', 'FI', 'RO', 'PA', 'MM', 'EG', 'FR', 'US', 'MN', 'LK', 'PT', 'TM', 'SI', 'VN', 'BE', 'VU', 'PL', 'CI', 'PH', 'MG', 'YE', 'RU'